In [1]:
import json
import pandas as pd

from dateutil.parser import parse as date_parse

In [2]:
data_path = './data/neo4j_graph/politicians_200907.json'

In [3]:
with open(data_path, 'r') as f:
    data = json.load(f)

In [4]:
following_relationship = data[0]
user_information = data[1]
category_information = data[2]

# to csv

Neo4jの「import tool」を使ってcsvファイルを読み込む場合、リレーションシップの始点と終点を指定するためのノードIDを用意する必要があります。ノードIDはすべてのノードに関して一意でなければなりません。ここではユーザノードはツィッターが用意しているユーザIDを用いるが、カテゴリとグループに関しては一意のIDを生成します。

## nodes

### category and group

カテゴリーとグループのリストを出します。

In [5]:
categories = list(set(c[0] for c in category_information))
groups = list(set([c[1] for c in category_information]))

後でリレーションシップを構築するためにカテゴリ・グループ名とIDの辞書を用意します

In [6]:
category_name_to_id = {c: 'category_{}'.format(i) for i,c in enumerate(categories)}
group_name_to_id = {g: 'group_{}'.format(i) for i,g in enumerate(groups)}

# category_df = pd.DataFrame(category_name_to_id.items(), columns=['categoryId:ID', 'name'])
# group_df = pd.DataFrame(group_name_to_id.items(), columns=['groupId:ID', 'name'])

category_df = pd.DataFrame(category_name_to_id.items(), columns=['name', ':ID'])
group_df = pd.DataFrame(group_name_to_id.items(), columns=['name', ':ID'])

In [7]:
category_df[':LABEL'] = 'CATEGORY'
category_df

,name,:ID,:LABEL
0,政治家・議員,category_0,CATEGORY


In [8]:
group_df[':LABEL'] = 'GROUP'
group_df.head()

,name,:ID,:LABEL
0,日本を元気にする会,group_0,GROUP
1,都知事選2016,group_1,GROUP
2,民進党,group_2,GROUP
3,日本維新の会,group_3,GROUP
4,日本の政党,group_4,GROUP


In [9]:
category_df.to_csv('./data/neo4j_graph/politician_category_node.csv', index=False)
group_df.to_csv('./data/neo4j_graph/politician_group_node.csv', index=False)

### user

In [10]:
def parse_user_feed(user_info):
    user_parse = {}
    for feed in ['id', 
                 'name', 
                 'screen_name', 
                 'location', 
                 'description', 
                 'protected', 
                 'followers_count', 
                 'friends_count', 
                 'listed_count', 
                 'created_at']:
        user_parse[feed] = user_info.get(feed)
        if feed == 'created_at' and user_info.get(feed):
            user_parse[feed] = date_parse(user_parse[feed])
        if 'count' in feed:
            if user_parse[feed]:
                user_parse[feed] = int(user_parse[feed])
    return user_parse

In [11]:
user_information_parsed = [parse_user_feed(u) for u in user_information]
users_with_information = set([u['id'] for u in user_information])

In [12]:
len(users_with_information)

347

In [13]:
users_in_relationship = []
for i, v in following_relationship.items():
    users_in_relationship += v

In [14]:
users_without_information = list(set(users_in_relationship) - set(users_with_information))

In [15]:
print(len(users_without_information), len(set(users_in_relationship)), len(users_with_information))
print(len(users_without_information) - len(set(users_in_relationship)), len(users_with_information))
print(set(users_without_information) & set(users_with_information))

282636 282982 347
-346 347
set()


In [16]:
user_information_parsed_no_info = [parse_user_feed({'id': u}) for u in users_without_information]

In [17]:
user_df = pd.DataFrame(user_information_parsed+user_information_parsed_no_info)

#### debug

In [18]:
fc = user_df.followers_count

In [19]:
fc_list = []
for x in fc.to_list():
    if pd.isnull(x):
        fc_list.append(None)
    else:
        fc_list.append(int(x))
    

In [20]:
pd.DataFrame(fc_list)

,0
0,2663.0
1,4107.0
2,2313.0
3,258141.0
4,6671.0
...,...
282978,NaN
282979,NaN
282980,NaN
282981,NaN


In [22]:
fc_list[:3]

[2663, 4107, 2313]

In [24]:
fc.apply(lambda x: int(x) if not pd.isnull(x) else x)

0           2663.0
1           4107.0
2           2313.0
3         258141.0
4           6671.0
            ...   
282978         NaN
282979         NaN
282980         NaN
282981         NaN
282982         NaN
Name: followers_count, Length: 282983, dtype: float64

In [39]:
fc.astype(int, errors='ignore')

0           2663.0
1           4107.0
2           2313.0
3         258141.0
4           6671.0
            ...   
282978         NaN
282979         NaN
282980         NaN
282981         NaN
282982         NaN
Name: followers_count, Length: 282983, dtype: float64

#### continue user

In [25]:
print(user_df.shape)

(282983, 10)


In [26]:
user_df.head()

,id,name,screen_name,location,description,protected,followers_count,friends_count,listed_count,created_at
0,99227667,米田 貴志,yonedatakashi,大阪府岸和田市,公明党岸和田市議会議員です。平成19年4月に初当選させて頂きました。市民を守る豊かな財政力を...,False,2663.0,2455.0,216.0,2009-12-25 05:45:31+00:00
1,141513987,前衆議院議員・いさか信彦（兵庫１区）,isakanobuhiko,兵庫県神戸市中央区八幡通4-2-14トロア神戸ビル4F,前衆議院議員・井坂信彦（神戸市中央区・灘区・東灘区）のtwitterです。本人およびスタッフ...,False,4107.0,2382.0,253.0,2010-05-08 08:23:24+00:00
2,911542923333820416,山川ゆりこ《立憲民主党 りっけん 埼玉3区,yamakawa_yuriko,埼玉 草加市・越谷市,埼玉３区（草加市・越谷市）の衆議院議員です。山川ゆりこは一貫して「武力によらない平和をつくる...,False,2313.0,104.0,89.0,2017-09-23 10:49:03+00:00
3,12392332,家入 一真,hbkr,,"金融包摂をつくる。CAMPFIRE代表取締役。BASE共同創業取締役。NOW, XIMERA...",False,258141.0,4020.0,4566.0,2008-01-18 09:01:26+00:00
4,43028151,吉田雄人,YoshidaYuto,,Glocal Government Relationz株式会社,False,6671.0,3117.0,530.0,2009-05-28 02:32:22+00:00


In [27]:
user_df.columns

Index(['id', 'name', 'screen_name', 'location', 'description', 'protected',
       'followers_count', 'friends_count', 'listed_count', 'created_at'],
      dtype='object')

# 問題
- datetime形式
- 文字コード

In [31]:
user_df.drop(columns=['created_at', 'description'])

,id,name,screen_name,location,protected,followers_count,friends_count,listed_count
0,99227667,米田 貴志,yonedatakashi,大阪府岸和田市,False,2663.0,2455.0,216.0
1,141513987,前衆議院議員・いさか信彦（兵庫１区）,isakanobuhiko,兵庫県神戸市中央区八幡通4-2-14トロア神戸ビル4F,False,4107.0,2382.0,253.0
2,911542923333820416,山川ゆりこ《立憲民主党 りっけん 埼玉3区,yamakawa_yuriko,埼玉 草加市・越谷市,False,2313.0,104.0,89.0
3,12392332,家入 一真,hbkr,,False,258141.0,4020.0,4566.0
4,43028151,吉田雄人,YoshidaYuto,,False,6671.0,3117.0,530.0
...,...,...,...,...,...,...,...,...
282978,133169129,None,None,None,None,NaN,NaN,NaN
282979,154140654,None,None,None,None,NaN,NaN,NaN
282980,108003321,None,None,None,None,NaN,NaN,NaN
282981,185597946,None,None,None,None,NaN,NaN,NaN


In [32]:
rename_dict = {
    'id': 'userId:ID',
    'protected': 'protected:boolean',
    'followers_count': 'followersCount:float',
    'friends_count': 'friendsCount:float',
    'listed_count': 'listedCount:float',
    'created_at': 'createdAt:date'
}

In [33]:
user_df_rename = user_df.rename(columns=rename_dict)
user_df_rename[':LABEL'] = 'USER'

In [34]:
user_df_rename

,userId:ID,name,screen_name,location,description,protected:boolean,followersCount:float,friendsCount:float,listedCount:float,createdAt:date,:LABEL
0,99227667,米田 貴志,yonedatakashi,大阪府岸和田市,公明党岸和田市議会議員です。平成19年4月に初当選させて頂きました。市民を守る豊かな財政力を...,False,2663.0,2455.0,216.0,2009-12-25 05:45:31+00:00,USER
1,141513987,前衆議院議員・いさか信彦（兵庫１区）,isakanobuhiko,兵庫県神戸市中央区八幡通4-2-14トロア神戸ビル4F,前衆議院議員・井坂信彦（神戸市中央区・灘区・東灘区）のtwitterです。本人およびスタッフ...,False,4107.0,2382.0,253.0,2010-05-08 08:23:24+00:00,USER
2,911542923333820416,山川ゆりこ《立憲民主党 りっけん 埼玉3区,yamakawa_yuriko,埼玉 草加市・越谷市,埼玉３区（草加市・越谷市）の衆議院議員です。山川ゆりこは一貫して「武力によらない平和をつくる...,False,2313.0,104.0,89.0,2017-09-23 10:49:03+00:00,USER
3,12392332,家入 一真,hbkr,,"金融包摂をつくる。CAMPFIRE代表取締役。BASE共同創業取締役。NOW, XIMERA...",False,258141.0,4020.0,4566.0,2008-01-18 09:01:26+00:00,USER
4,43028151,吉田雄人,YoshidaYuto,,Glocal Government Relationz株式会社,False,6671.0,3117.0,530.0,2009-05-28 02:32:22+00:00,USER
...,...,...,...,...,...,...,...,...,...,...,...
282978,133169129,None,None,None,None,None,NaN,NaN,NaN,NaT,USER
282979,154140654,None,None,None,None,None,NaN,NaN,NaN,NaT,USER
282980,108003321,None,None,None,None,None,NaN,NaN,NaN,NaT,USER
282981,185597946,None,None,None,None,None,NaN,NaN,NaN,NaT,USER


In [57]:
user_df_rename.to_csv('./data/neo4j_graph/politician_user_node.csv', index=False, quotechar='"')

## relationship

### category group

In [29]:
category_group_relationship = [
    [category_name_to_id[r[0]], group_name_to_id[r[1]]] 
    for r in set([(c[0], c[1]) for c in category_information])
]

In [30]:
category_group_df = pd.DataFrame(category_group_relationship, columns=[':END_ID', ':START_ID'])

In [31]:
category_group_df[':TYPE'] = 'BELONG'

In [30]:
category_group_df.to_csv('./data/neo4j_graph/politician_category_group_relationship.csv', index=False)

NameError: name 'category_group_df' is not defined

### category user

In [33]:
user_screen_name_id_dict = {u['screen_name']: u['id'] for u in user_information_parsed}

In [34]:
category_information

[['政治家・議員', '日本のこころを大切にする党', 'miosugita'],
 ['政治家・議員', '日本のこころを大切にする党', 'nakayamanariaki'],
 ['政治家・議員', '日本のこころを大切にする党', 'nipponkokoro'],
 ['政治家・議員', '日本のこころを大切にする党', 'masashi_nakano'],
 ['政治家・議員', '日本維新の会', 'hiroyoshimura'],
 ['政治家・議員', '日本維新の会', 'gogoichiro'],
 ['政治家・議員', '日本維新の会', 'adachiyasushi'],
 ['政治家・議員', '日本維新の会', 'maruyamahodaka'],
 ['政治家・議員', '日本維新の会', 'loveyassy'],
 ['政治家・議員', '日本維新の会', 'fujimaki_takesi'],
 ['政治家・議員', '日本維新の会', 'osaka_ishin'],
 ['政治家・議員', '日本維新の会', 'fumi_fuji'],
 ['政治家・議員', '日本維新の会', 'asalogue'],
 ['政治家・議員', '日本維新の会', 'toru_azuma'],
 ['政治家・議員', '日本維新の会', 'katayama1122'],
 ['政治家・議員', '日本維新の会', 'mikioshimoji'],
 ['政治家・議員', '日本維新の会', 'uranoyasuto'],
 ['政治家・議員', '日本維新の会', 'ishii_ishin'],
 ['政治家・議員', '日本維新の会', 'CoTakayuki'],
 ['政治家・議員', '日本維新の会', 'izki_toyama'],
 ['政治家・議員', '日本維新の会', 'OzawaSakihito'],
 ['政治家・議員', '日本維新の会', 'katayamayusuke'],
 ['政治家・議員', '日本維新の会', 'keemiki'],
 ['政治家・議員', '日本維新の会', 'matsunamikenta'],
 ['政治家・議員', '日本維新の会', 'YWatanabeOffice'],
 ['政治